# Вариационная нижняя граница

Цели:

* Вспомнить, что такое KL-дивергенция и как она выводится
* Получить представление о том, как используется ELBO в VAE

Содержание:

* [KL-дивергенция](#kld)
  * Вывод формулы
  * Прямая и обратная KL-дивергенция
* [Формула Байеса](#bayes)
* [ELBO](#elbo)
* [Обучение VAE](#vae)
  * Трюк репараметризации
  * Реализация


Ссылки:

* [VAE](https://www.youtube.com/watch?v=9_eZHt2qJs4) and [ELBO](https://www.youtube.com/watch?v=IXsA5Rpp25w) clearly explained
* ["From Autoencoder to Beta-VAE" страничка в блоге Lilian Weng](https://lilianweng.github.io/posts/2018-08-12-vae/)
* [Variational inference & deep learning. A new synthesis, Kingma, D.P. PhD Thesis](https://pure.uva.nl/ws/files/17891313/Thesis.pdf) - от автора публикации про VAE
* [Дмитрий Кропотов | Вариационный авто-кодировщик. Летняя школа AIRI 2022](https://www.youtube.com/watch?v=wRHUc44PJuE)
* Дмитрий Ветров. Летняя школа AIRI 2022. Состоит из двух частей:
  * [Введение в байесовский подход к теории вероятностей. Основы байесовского вывода.](https://www.youtube.com/watch?v=RZGdMiBOYwk)
  * [Вариационный вывод. Дважды-стохастический вывод. Вариационный авто-кодировщик.](https://www.youtube.com/watch?v=QEBKaM3z1sM)

<a name="kld"></a>
## KL-дивергенция

KL-дивергенция получила своё название по имени двух учёных - Solomon **K**ullback и Richard **L**eibler, она также называется дивергенция Кульбака-Лейблера.

Она нужна для того, чтобы измерить, насколько одно распределение вероятности, отличается от другого, базового. Часто используется, когда мы хотим аппроксимировать какое-то сложное распределение более простым.

Пусть $X = \{x_1, x_2, ..., x_n\}$ - случайная величина (СВ).

Обозначим $p_\theta$ и $q_ϕ$ - наши распределения, которые мы хотим сравнить, где $\theta$ и $\phi$ - параметры распределения.

Тогда $p_\theta(x_i)$ и $q_ϕ(x_i)$ - вероятности конкретных значений.

Вероятности могут быть очень малыми числами, поэтому часто работают с логарифмами вероятностей.

In [ ]:
import numpy as np

# let's check for some small value
p = 1e-6
print(f'{p:.6f} -> {np.log(p):.6f}')

0.000001 -> -13.815511


### Вывод формулы

Если мы хотим измерить отличие распределений, то можно просто посчитать разницу

$log\ p_\theta(x_i) - log\ q_ϕ(x_i) = log\Big[\frac{p_\theta(x_i)}{q_ϕ(x_i)}\Big]$

Выражение под скобками называется коэффициент правдоподобия (**likelihood ratio**).

Но это разница между какими-то конкретными значениями, а у нас распределения, поэтому мы хотим знать мат. ожидание разницы.

Мат. ожидание дискретной СВ считается как:

$\mathbb{E_{p_\theta}}[X] = \displaystyle\sum_{i=1}^{\infty}x_ip_\theta(x_i)$

Также можно посчитать мат. ожидание значения функции от дискретной СВ (оно нам позже пригодится):

$\mathbb{E_{p_\theta}}[h(X)] = \displaystyle\sum_{i=1}^{\infty}h(x_i)p_\theta(x_i)$

Для непрерывной СВ:

$\mathbb{E_{p_\theta}}[h(X)] = \displaystyle\int{h(x)p_\theta(x)\ \mathrm{d}x}$

Логарифм коэффициента правдоподобия не что иное, как функция от СВ, поэтому чтобы получить его мат. ожидание, достаточно посчитать:

$\mathbb{E}\Big[log\frac{p_\theta(x)}{q_ϕ(x)}\Big] = \displaystyle\sum_{i=1}^{\infty}p_\theta(x_i)log\Big[\frac{p_\theta(x_i)}{q_ϕ(x_i)}\Big]$

Для непрерывной СВ соответственно:

$\mathbb{E}\Big[log\frac{p_\theta(x)}{q_ϕ(x)}\Big] = \displaystyle\int{p_\theta(x)log\Big[\frac{p_\theta(x)}{q_ϕ(x)}\Big]\ \mathrm{d}x}$

Это и есть KL-дивергенция:

$D_{KL}(p_\theta \parallel q_ϕ)= \displaystyle\int{p_\theta(x)log\Big[\frac{p_\theta(x)}{q_ϕ(x)}\Big]\ \mathrm{d}x}$

Но у нас сумммирование по бесконечному набору значений и интеграл по всей действительной оси, как это вычислить на практике?

Закон больших чисел говорит, что среднее СВ при большом количестве наблюдений стремится к мат. ожиданию.

Перепишем формулу (теперь она одинаковая для дискретных и непрерывных СВ):

$\mathbb{E}\Big[log\frac{p_\theta(x)}{q_ϕ(x)}\Big] \approx \frac1N\displaystyle\sum_{i=1}^Np_\theta(x_i)log\Big[\frac{p_\theta(x_i)}{q_ϕ(x_i)}\Big]$

### Прямая и обратная KL-дивергенция

KL-дивергенция несимметрична и для $q_\phi$ она будет считаться так:

$D_{KL}(q_ϕ \parallel p_\theta)= \displaystyle\int{q_\phi(x)log\Big[\frac{q_ϕ(x)}{p_\theta(x)}\Big]\ \mathrm{d}x}$

Значение очевидно будет другим для тех же $p_\theta$ и $q_\phi$, поэтому она не является метрикой:

$D_{KL}(p_\theta \parallel q_ϕ) \ne D_{KL}(q_ϕ \parallel p_\theta)$

Кстати левая часть называется Forward KL, правая - Reverse KL.

Интересно посмотреть на разницу между ними, когда мы пытаемся аппроксимировать бимодальное базовое распределение унимодальным:

* дивергенция меньше, если приближение лучше в точках, где базовое распределение имее большие значения, поэтому при обратной $Q$ соответствует моде распределения $P$
* при прямой $Q$ стремится покрыть базовое распределение $P$ целиком

<figure>
<img src="https://lilianweng.github.io/posts/2018-08-12-vae/forward_vs_reversed_KL.png" alt="Forward vs Reverse KL" style="width:100%">
<figcaption align = "center"> Image credit: <a href="https://blog.evjang.com/2016/08/variational-bayes.html">Eric Jang's blog</a></figcaption>
</figure>


Нас интересует скорее мода, это соответствует максимуму апостериорного распределения.

При оценке плотности распределения и в вариационном выводе используется именно Reverse KL, а Forward KL неявно используется в ML, в частности когда функция потерь - кросс-энтропия.

<a name="bayes"></a>
## Формула Байеса

$\text{posterior} = \frac{\text{likelihood}\ \times \text{prior}}{\text{evidence}}$

Формула Байеса для получения апостериорного распределения латентной переменной $z$:

$p_\theta(z | x) = \frac{p_\theta(x | z)p_\theta(z)}{p_\theta(x)}$, где

* $p_\theta(z | x)$ - апостериорное распределение после того, как мы пронаблюдали (posterior) $x$
* $p_\theta(x | z)$ - правдоподобие данных при условии $z$ (likelihood)
* $p_\theta(z)$ - априорное распределение до того, как мы пронаблюдали $x$ (prior)
* $p_\theta(x)$ - нормировочная константа или обоснованность (evidence)

В знаменателе

$p_\theta(x) = \displaystyle\int{p_\theta(x | z)p_\theta(z)\ \mathrm{d}x}$

находится неберущийся для многомерных пространств интеграл.



<a name="elbo"></a>
## ELBO

Итак у нас есть какое-то сложное распределение $p(x|\theta)$ (запись $p_\theta(x)$ эквивалентна) и мы хотим максимизировать логарифм правдоподобия $log\ p_θ(x)$ по параметрам θ.

Вводим латентную переменную $z$, тогда

$log\ p_θ(x) = \displaystyle\int{log\ p_θ(x, z)\mathrm{d}z} = \displaystyle\int{log\ p_θ(z | x)p_θ(x)\ \mathrm{d}z}$

Интеграл этот неберущийся в большинстве случаев, а сложность численного интегрирования растёт экспоненциально с увеличением размерности пространства.

Какой выход - использовать вариационный вывод, который предлагает нам для вычисления апостериорного распределения $p_\theta(z | x)$ использовать аппроксимацию более простым расеределением $q_\phi(z | x)$, для которого мы знаем как делать апостериорный вывод и из которого будет легко сэмплировать. И подберём параметры $\phi$ так, чтобы оно было близко к истинному.

NOTE: В вариационном исчислении мы ищем функцию, при которой функционал достигает максимального значения. Функционал - это функция от функций. А мы ищем в семействе распределений то, которое даёт нам максимум правдоподобия.

Будем подбирать параметры $\phi$ путём оптимизации функции потерь. В качестве меры непохожести между истинным распределением и аппроксимацией будем использовать KL-дивергенцию.

Задача превращается в задачу минимизации обратной KL-дивергенции между p и q (мат. ожидание коэффициента правдоподобия):

$D_{KL}(q_ϕ∥p_θ) = \mathbb{E}_{q_\phi}\Big[log\frac{q_ϕ(z|x)}{p_\theta(z|x)}\Big]$


Но как вычислить знаменатель?

Разделим проблему на части:

$\mathbb{E}_{q_\phi}\Big[log\frac{q_ϕ(z|x)}{p_\theta(z|x)}\Big] = \mathbb{E}_{q_\phi}[log\ {q_ϕ(z|x)}] - \mathbb{E}_{q_\phi}[log\ p_\theta(z|x)]$

Распишем второй член по формуле условной вероятности:

$ = \mathbb{E}_{q_\phi}[log\ {q_ϕ(z|x)}] - \mathbb{E}_{q_\phi}\Big[log\frac{p_θ(z,x)}{p_θ(x)}\Big]$

Воспользуемся свойствами логарифма:

$ = \mathbb{E}_{q_\phi}[log\ {q_ϕ(z|x)}] - \mathbb{E}_{q_\phi}[log\ p_θ(z,x)] + \mathbb{E}_{q_\phi}[log\ p_θ(x)]$

Распишем мат. ожидание в третьем члене:

$ = \mathbb{E}_{q_\phi}[log\ {q_ϕ(z|x)}] - \mathbb{E}_{q_\phi}[log\ p_θ(z,x)] + \displaystyle\int{q_ϕ(z|x)log\ p_θ(x)\mathrm{d}z}$

Вынесем константу за интеграл:

$ = \mathbb{E}_{q_\phi}[log\ {q_ϕ(z|x)}] - \mathbb{E}_{q_\phi}[log\ p_θ(z,x)] + log\ p_θ(x)\displaystyle\int{q_ϕ(z|x)\mathrm{d}z}$

Так как интеграл функции плотности вероятности равен 1, получим:

$ = \mathbb{E}_{q_\phi}[log\ {q_ϕ(z|x)}] - \mathbb{E}_{q_\phi}[log\ p_θ(z,x)] + log\ p_θ(x)$

Третий компонент - логарифм маргинального правдоподобия (marginal log likelihood) или логарифм "обоснованности" (evidence). Маргинальная - в том смысле, что мы уже проинтегрировали по скрытой переменой $z$. Ещё его называют логарифм неполного правдоподобия.

Переносим логарифм правдоподобия влево:

$log\ p_θ(x) = - \mathbb{E}_{q_\phi}[log\ {q_ϕ(z|x)}] + \mathbb{E}_{q_\phi}[log\ p_θ(z,x)] + D_{KL}(q_ϕ∥p_θ) \quad\quad(1)$

При этом KL-дивергенция неотрицательная:

$D_{KL}(q_ϕ∥p_θ) ≥ 0$

Отсюда получаем, что

$log\ p_θ(x) ≥ - \mathbb{E}_{q_\phi}[log\ {q_ϕ(z|x)}] + \mathbb{E}_{q_\phi}[log\ p_θ(z,x)]$

Правая часть выражения называется вариационной нижней границей (Variational Lower Bound или Evidence Lower BOund).

Правая часть в формуле (1) не зависит от $z$, поэтому если мы максимизируем ELBO, мы неявно минимизируем KL-дивергенцию в формуле.

Упростим ещё:

$ELBO = - \mathbb{E}_{q_\phi}[log\ {q_ϕ(z|x)}] + \mathbb{E}_{q_\phi}[log\ p_θ(z,x)]$

$= - \mathbb{E}_{q_\phi}[log\ {q_ϕ(z|x)}] + \mathbb{E}_{q_\phi}[log\ p_θ(x|z)]  + \mathbb{E}_{q_\phi}[log\ p_θ(z)]$

$= \mathbb{E}_{q_\phi}[log\ p_θ(x|z)] - \mathbb{E}_{q_\phi}[log\ {q_ϕ(z|x)}] + \mathbb{E}_{q_\phi}[log\ p_θ(z)]$

$= \mathbb{E}_{q_\phi}[log\ p_θ(x|z)] - \mathbb{E}_{q_\phi}\Big[log\frac{{q_ϕ(z|x)}}{p_θ(z)}\Big]$

Первый член выражения говорит, что это мат. ожидание ошибки аппроксимации (expected reconstruction error), а вторая - KL-дивергенция между аппроксимированным апостериорным распределением и априорным распределением.

Будем использовать $-ELBO$ в качестве функции потерь.

<a name="vae"></a>
## Обучение VAE

VAE состоит из двух частей:

* энкодер, который отображает сложное входное распределение в многомерное нормальное - аппроксимирует распределение $q_ϕ(z|x)$ по параметрам $ϕ$
* декодер, отображающий нормальное распределение обратно в исходное - аппроксимирует $p_θ(x|z)$ по параметрам $θ$

Задача - найти минимум целевой функции $\underset{θ, ϕ}{\mathrm{argmin}} \ \mathcal L = \underset{θ, ϕ}{\mathrm{argmin}} -\mathbb{E}_{q_\phi}[log\ p_θ(x|z)] + \mathbb{E}_{q_\phi}\Big[log\frac{{q_ϕ(z|x)}}{p_θ(z)}\Big]$.

Первый член с мат. ожиданием в выражении выше требует сэмплирования из многомерного нормального распределения. Сэмплирование является стохастическим процессом и через него не будут распространяться обратно градиенты.

### Трюк репараметризации

Репараметризуемое распределение - то, которое можно выразить как детерминированную функцию от непараметрического распределения.

$z ∼ q_ϕ(z|x) = \mathcal N(z; μ, diag(σ^2))$

$z = μ + σ \odot ϵ, где \ ϵ \sim \mathcal N(0, I), \odot - поэлементное \ произведение$

<figure>
<img src="https://lilianweng.github.io/posts/2018-08-12-vae/reparameterization-trick.png" alt="Reparametrisation trick" style="width:100%">
<figcaption align = "center"> Image credit: <a href="https://lilianweng.github.io/posts/2018-08-12-vae/">Lilian Weng's blog</a></figcaption>
</figure>


### Реализация

На практике функцию потерь вычисляют как сумму BCE

$BCE(y,\hat y)=−y\cdot log(\hat y)−(1−y)\cdot log(1−\hat y)$

и KL-дивергенции распределения $q_ϕ(z|x)$ со стандартным нормальным, которая выступает в роли регуляризатора.

При сравнении нормального распределения со стандартным она расписывается следующим образом:

$D_{KL}(\mathcal{N}(\mu,\upsilon)\parallel \mathcal{N}(0,1))=\frac{1}{2}\displaystyle\sum_{i=1}^d\upsilon_i - log(\upsilon_i) - 1 + \mu_i^2$

Оптимизируется стохастическим градиентным спуском.